In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision as tv

from torchvision.datasets import MNIST
from tqdm import tqdm

Загружаем датасет и проводим трансформации над ним: сначала трансформируем каждую картинку в тензор, затем нормализуем с указанными параметрамми мат.ожидания и среднеквадратического отклонения.

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

batch_size=128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

Строим классификатор. Данный классификатор представляет собой известную нейросеть LeNet, с небольшими изменениями, чтобы подстроиться под разрешение картинки в MNIST. Также добавлены дополнительные полносвязные слои в конец нейросети, т.к. это позволило добиться точности более 99%.

In [26]:
class Classificator(nn.Module):
    def __init__(self):
        super(Classificator, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=9) # after that step: 20x20x6
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # after that step: 10x10x6
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3) # after that step: 8x8x16
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # after that step: 4x4x16
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=32)
        self.fc4 = nn.Linear(in_features=32, out_features=16)
        self.fc5 = nn.Linear(in_features=16, out_features=8)
        self.fc6 = nn.Linear(in_features=8, out_features=10)

        self.leaky_relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.pool1(self.leaky_relu(self.conv1(x)))
        x = self.pool2(self.leaky_relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = self.leaky_relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.leaky_relu(self.fc3(x))
        x = self.leaky_relu(self.fc4(x))
        x = self.leaky_relu(self.fc5(x))
        x = self.fc6(x)

        return x


Создаём объект нейросети, в качестве функции потерь выбираем кросс-энтропию, в качестве оптимизирующего алгоритма - Adam. Важно отметить, что реализация nn.CrossEntropyLoss() в Pytorch совмещает в себе softmax activation function и neagative log likelihood в единой функции, поэтому явно слой с softmax в нейросети не используется. Также инициализируем гиперпараметры нейросети, затем начинаем обучение.

In [31]:
clf = Classificator()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
num_epochs = 100
optimizer = torch.optim.Adam(clf.parameters(), lr=learning_rate)

for epoch in tqdm(range(num_epochs)):
    for data in train_loader:
        img, labels = data
        optimizer.zero_grad()
        output = clf(img)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

  1%|          | 1/100 [00:18<30:46, 18.65s/it]

Epoch [1/100], Loss: 0.3374


  2%|▏         | 2/100 [00:34<27:32, 16.87s/it]

Epoch [2/100], Loss: 0.1665


  3%|▎         | 3/100 [00:57<31:56, 19.76s/it]

...

Epoch [98/100], Loss: 0.0070


 99%|█████████▉| 99/100 [29:07<00:17, 17.29s/it]

Epoch [99/100], Loss: 0.0019


100%|██████████| 100/100 [29:20<00:00, 17.60s/it]

Epoch [100/100], Loss: 0.0005


Осталось протестировать полученную модель. Как видно, за 100 итераций обучения с такой архитектурой нейросети поулчилось достичь точности в 99.15%.

In [39]:
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = clf(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy on the test set:  {(100 * correct / total)} %')

Accuracy on the test set:  99.15 %
